<a href="https://colab.research.google.com/github/PBustamante65/Python/blob/main/2023_Cars_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 7.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df_predict = pd.read_csv('/content/drive/MyDrive/Data Science/2023 Cars/2023Cars_no_price.csv')

In [ ]:
df_predict.columns

Index(['Car Make', 'Car Model', 'Year', 'Body Type', 'Color Options',
       'Fuel Type', 'Engine Size (L)', 'Horsepower', 'Torque (Nm)',
       'Transmission Type', 'Acceleration (0-60 mph)', 'Top Speed (mph)',
       'Mileage (MPG)', 'Safety Features', 'Entertainment Features',
       'Interior Features', 'Exterior Features', 'Customer Ratings',
       'Sales Figures (Units Sold)', 'id'],
      dtype='object')

In [ ]:
#We rename the columns because some, become clumsy to work with, especially with all those spaces
column_name_changes = {
    ' Price ($) ': 'Price',
    ' Exterior Features                                           ': 'Exterior Features',
    ' Interior Features                                ': 'Interior Features',
    ' Car Make ': 'Car Make',
    ' Car Model   ': 'Car Model',
    ' Year ': 'Year',
    ' Body Type ': 'Body Type',
    ' Color Options                 ': 'Color Options',
    ' Fuel Type ': 'Fuel Type',
    'Engine Size (L)': 'Engine Size(L)',
    ' Horsepower ' : 'Horsepower',
    ' Torque (Nm) ' : 'Torque (Nm)',
    ' Transmission Type ': 'Transmission Type',
    ' Acceleration (0-60 mph) ': 'Acceleration (0-60 mph)',
    ' Top Speed (mph) ': 'Top Speed (mph)',
    ' Mileage (MPG) ': 'Mileage (MPG)',
    ' Safety Features                                      ': 'Safety Features',
    ' Entertainment Features               ': 'Entertainment Features',
    ' Customer Ratings ': 'Customer Ratings',
    ' Sales Figures (Units Sold)': 'Sales Figures (Units Sold)'

}

df_predict.rename(columns=column_name_changes, inplace=True)



In [ ]:

#We also drop the comma and convert into float
df_predict['Sales Figures (Units Sold)'] = df_predict['Sales Figures (Units Sold)'].str.replace(',', '').astype(float)

#In this one, we drop the 'out of five (x/5)' part of the rating, might change the columns name to better reflect this in the future
df_predict['Customer Ratings'] = df_predict['Customer Ratings'].str.split('/').str[0]

#We drop the word seconds from the column and convert it to a float
df_predict['Acceleration (0-60 mph)'] = df_predict['Acceleration (0-60 mph)'].str.replace(' seconds', '').astype(float)

#The following was to drop an abreviation that appeared as (est.) when it came to electric cars
df_predict['Horsepower'] = df_predict['Horsepower'].str.split('(').str[0]
df_predict['Torque (Nm)'] = df_predict['Torque (Nm)'].str.split('(').str[0]
df_predict['Top Speed (mph)'] = df_predict['Top Speed (mph)'].str.split('(').str[0]
df_predict['Mileage (MPG)'] = df_predict['Mileage (MPG)'].str.split('(').str[0]

In [ ]:
#After seeing the previous graph, we could see something was a miss, so after further diggin, we found 2 entries that were wrong, so we went ahead and fix them
#Here we modify row 103, and 139, after some digging, we found that the info in some rows, were actually in another one, so, we set out to put everything in its place
rows_to_modify = [80, 147]
columns_to_modify = ['Customer Ratings', 'Sales Figures (Units Sold)', 'Mileage (MPG)', 'Entertainment Features', 'Safety Features']
new_values = [4.9, 900, 110, '10.2-inch Infotainment Display', 'ABS, Airbags, HD Rear Vision Camera, Lane Keep Assist with Lane Departure Warning']

for row_index in rows_to_modify:
    for col_index, column in enumerate(columns_to_modify):
        new_value = new_values[col_index]
        df_predict.loc[row_index, column] = new_value




In [ ]:
rows_to_modify = [80]
columns_to_modify = ['Exterior Features', 'Interior Features']
new_values = ['LED Headlights, Roof Rails, 17-inch Alloy Wheels', 'HD Rear Wireless Smartphone Charging']

for row_index in rows_to_modify:
    for col_index, column in enumerate(columns_to_modify):
        new_value = new_values[col_index]
        df_predict.loc[row_index, column] = new_value


rows_to_modify = [147]
columns_to_modify = ['Interior Features']
new_values = ['Leather Seats, Keyless Entry, Wireless Smartphone Charging']

for row_index in rows_to_modify:
    for col_index, column in enumerate(columns_to_modify):
        new_value = new_values[col_index]
        df_predict.loc[row_index, column] = new_value

In [ ]:
#We convert Top Speed to a float
df_predict['Top Speed (mph)'] = df_predict['Top Speed (mph)'].str.replace(' mph', '').astype(float)

#Convert Mileage to float
df_predict['Mileage (MPG)'] = df_predict['Mileage (MPG)'].str.replace(' MPG', '').astype(float)




In [ ]:
#We convert Torque to float and drop the empty values of some rows
df_predict['Torque (Nm)'] = df_predict['Torque (Nm)'].str.replace(' N/A         ', '').str.replace(' Nm', '').str.strip()
df_predict = df_predict[df_predict['Torque (Nm)'] != '']  # Remove rows with empty values
df_predict['Torque (Nm)'] = df_predict['Torque (Nm)'].astype(float)

#Convert Horsepower into float
df_predict['Horsepower'] = df_predict['Horsepower'].astype(float)

<ipython-input-141-87b823d63a33>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict['Torque (Nm)'] = df_predict['Torque (Nm)'].astype(float)
<ipython-input-141-87b823d63a33>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict['Horsepower'] = df_predict['Horsepower'].astype(float)


In [ ]:
#Once found, we can see its actually not a lot of values, so it makes no harm if we drop em for a cleaner workspace
df_predict = df_predict.drop(df_predict.loc[df_predict['Mileage (MPG)'].isnull()].index)
df_predict = df_predict.drop(df_predict.loc[df_predict['Exterior Features'].isnull()].index)
df_predict = df_predict.drop(df_predict.loc[df_predict['Interior Features'].isnull()].index)

#We dropped the outliers

rows_to_drop = [148, 40, 107,135,41,88,4]
df_predict = df_predict.drop(rows_to_drop)

#Log transform for slighly right skewness
df_predict['NormMileage'] = np.log(df_predict['Mileage (MPG)'])

#Exponential transformation (X^2) for slightly right skewness
df_predict['NormAcceleration'] = np.power(df_predict['Acceleration (0-60 mph)'], 2)


#Log transformation for right skewness
df_predict['NormTorque'] = np.log(df_predict['Torque (Nm)'])


#Log transformation for left skewness
df_predict['NormHorsepower'] = np.log(df_predict['Horsepower'])




In [ ]:
#We encode and separate our data to be used for predictions
car_features = ['NormMileage', 'NormAcceleration', 'NormTorque', 'NormHorsepower', 'Engine Size(L)', 'Year']
columns_to_encode = ['Body Type', 'Color Options', 'Fuel Type', 'Transmission Type', 'Car Make', 'Car Model', 'Safety Features', 'Entertainment Features', 'Interior Features', 'Exterior Features' ]

df_predict_en = pd.get_dummies(df_predict, columns=columns_to_encode, prefix='Encode')
en_columns = df_predict_en.filter(like='Encode')
selected_columns = car_features + en_columns.columns.tolist()

X = df_predict_en[selected_columns]

In [ ]:
#We import our previouslt trained model
from catboost import CatBoostRegressor

catboost_model = CatBoostRegressor()
catboost_model.load_model('/content/drive/MyDrive/Data Science/2023 Cars/carboost_model.cbm')

In [ ]:
#We do the prediction and save it on a new variable
predictions = catboost_model.predict(X)

In [ ]:
#This variable is still normalized, so we have to do an exponential transformation e^x to obtain our original price scale
normalized_predictions = np.array([predictions])

original_pvalues = np.exp(normalized_predictions)

print(original_pvalues)

[[32349.94187463 25000.01448213 28500.02774419 42600.07025937
  38949.94580057 22495.03548655 33899.99443331 45450.0598835
  37510.05512767 26995.04620102 45899.94118773 28699.95491157
  36085.01213105 21820.00565231 22950.00348705 34749.9394495
  37774.86434237 21699.98606608 33419.96594394 26170.00759757
  19489.97539975 21489.9864416  22594.98527138 67590.06325281
  26649.95442978 75589.96255778 54499.98225821 41049.94649192
  27500.01824902 28570.04601014 53249.94135701 51850.07981161
  42700.04718099 43199.9614892  35600.0394231  37775.0180533
  38500.039626   31945.04159405 43899.93881703 27849.97183662
  36750.04181656 62699.91756402 33899.99443331 28450.0336296
  28949.99728193 20400.01006574 20400.01006574 47699.99027102
  23900.02912299 42850.00891053 47699.99027102 25000.00408062
  29350.02758721 25000.00408062 17599.99429556 39899.93837167
  17599.99429556 21820.00565231 51249.96078764 21820.00565231
  31449.96286259 26170.00759757 26170.00759757 30894.94733366
  21489.9864

In [ ]:
original_pvalues = original_pvalues.ravel()
df_pvalues = pd.DataFrame({'Price_predictions': original_pvalues})

df_with_predictions = pd.concat([df_predict, df_pvalues], axis=1)


In [ ]:
df_with_predictions.head()